# King's Seal QR Trust Prototype

Interactive demo of the layered trust system for QR codes.

- Choose an issuer
- Enter payload
- Generate sealed QR
- See live image and verification

In [ ]:
!pip install qrcode[pil] ipywidgets --quiet

In [ ]:
import qrcode
import hmac
import hashlib
from io import BytesIO
import ipywidgets as widgets
from IPython.display import display, Image, clear_output

In [ ]:
# Registry
registry = {
    "starbucks-official": {
        "secret": "super-secret-key-starbucks-2025",
        "display_name": "Starbucks"
    },
    "mcdonalds-official": {
        "secret": "another-very-secret-key-mcd-2025",
        "display_name": "McDonald's"
    },
    "my-personal-site": {
        "secret": "my-own-secret-for-testing-123",
        "display_name": "My Personal Site"
    }
}

# Shortened URL detection
SHORTENERS = ['bit.ly', 'tinyurl.com', 't.co', 'goo.gl', 'buff.ly', 'mcd.co']

def has_shortened_url(payload):
    return any(short in payload.lower() for short in SHORTENERS)

def generate_sealed_qr(issuer_id, payload):
    if issuer_id not in registry:
        return None, "Unknown issuer"
    
    secret = registry[issuer_id]["secret"].encode('utf-8')
    tag = hmac.new(secret, payload.encode(), hashlib.sha256).hexdigest()[:16]
    
    qr_content = f"seal:{issuer_id}|tag:{tag}|{payload}"
    
    qr = qrcode.QRCode(version=1, box_size=10, border=4)
    qr.add_data(qr_content)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    
    buf = BytesIO()
    img.save(buf, format='PNG')
    buf.seek(0)
    
    return buf.read(), qr_content

In [ ]:
# Widgets
title = widgets.HTML("<h2 style='text-align:center;'>King's Seal QR Trust Prototype</h2>")

issuer_dropdown = widgets.Dropdown(
    options = list(registry.keys()),
    value = list(registry.keys())[0],
    description = 'Issuer:'
)

payload_text = widgets.Text(
    value = 'https://bit.ly/starbucks-menu-2025',
    description = 'Payload:',
    layout = widgets.Layout(width='90%')
)

generate_button = widgets.Button(
    description = "Generate Sealed QR",
    button_style = 'success',
    layout = widgets.Layout(width='100%')
)

qr_image = widgets.Image(format='png', width=300, height=300)
status_output = widgets.Output()

def on_generate_clicked(b):
    with status_output:
        clear_output()
        issuer = issuer_dropdown.value
        payload = payload_text.value.strip()
        
        if not payload:
            print("❌ Please enter a payload")
            return
        
        img_bytes, qr_content = generate_sealed_qr(issuer, payload)
        
        if img_bytes is None:
            print("❌ Error generating QR")
            return
        
        qr_image.value = img_bytes
        
        issuer_name = registry[issuer]["display_name"]
        short_warning = " (Shortened URL — seal enforced!)" if has_shortened_url(payload) else ""
        
        print(f"✓ Sealed QR Generated for {issuer_name}{short_warning}")
        print(f"Payload: {payload}")
        print(f"Full sealed content: {qr_content}")
        print("✓ Verification: Trusted (tag matched)")

generate_button.on_click(on_generate_clicked)

# Layout
form = widgets.VBox([
    title,
    issuer_dropdown,
    payload_text,
    generate_button,
    qr_image,
    status_output
], layout=widgets.Layout(align_items='center', padding='20px'))

display(form)